# Max or Min outcomes. Sales. Let's plug it in

### How much can we charge to increase revenue. BY HAND
### https://www.youtube.com/watch?v=GMGNoRcZEwM&t=73s

In [2]:
subscription = 500 # per year
member_sales_per_week = 180
# for every $5 discount = 3 more sales. No overhead increase...

find sales as a function of price. get y intercept.
Price 500 - 5. When does this = 0?

In [5]:
# When does $500 - $5 = 0?

In [7]:
# 500-500-5d=0-500
# -5d=-500
# /by -5
# d = 100. When d is 100, x is 0. This is how we get y intercept.

Now substitute this into sales. intercept = 180 + 3d

In [9]:
# y_intercept = 480
# when price is $0, expected price is 480 lolz

In [10]:
# slope = Change in Y / Change in X
# Sales 180+3d, Price 500 - 5d
# slope is +3 / -5 = -0.6
# Sales = 480(yint) - 0.6*price

In [13]:
# Revenue = (480 - 0.6*price)(price) or 480*price - 0.6*price**2

In [16]:
# get derivitive
# 480*price - 0.6*price**2
# derivitive of 480*price is 480 ... because we know that's the price. price was x.
# -0.6*price**2 is 0.6*2*price.. is 1.2*price
# now solve for 0! Why? because that's giving us where Y is at the max. Where is slope 0

In [18]:
# solve for 0.
# When does 480 - 1.2*price = 0?
# -480 by each side, divide by -1.2... price = 400. more income if we charge 400 instead.

In [19]:
# how many sales can we get?
# take 480 - 0.6*400, 240 total. 240 new subscriptions per week.

# apparently we can use Scipy for optimization

In [21]:
import scipy

In [24]:
print(scipy.__file__)

/Users/louisrossi/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py


In [34]:
from pathlib import Path
import numpy as np
from scipy.cluster.vq import whiten, kmeans, vq
import pandas as pd

In [60]:
data = Path("SMSSpamCollection copy").read_text()
data = data.strip()
data = data.split("\n")

In [58]:
pd.DataFrame(data)

,0
0,"ham\tGo until jurong point, crazy.. Available ..."
1,ham\tOk lar... Joking wif u oni...
2,spam\tFree entry in 2 a wkly comp to win FA Cu...
3,ham\tU dun say so early hor... U c already the...
4,"ham\tNah I don't think he goes to usf, he live..."
...,...
5569,spam\tThis is the 2nd time we have tried 2 con...
5570,ham\tWill ü b going to esplanade fr home?
5571,"ham\tPity, * was in mood for that. So...any ot..."
5572,ham\tThe guy did some bitching but I acted lik...


In [ ]:
#### create numpy array rows = number of texts.
#### use to associate number of digits in the message to see if spam or not

In [63]:
# scipy needs numpy arrays as input so we cant use collections
digit_counts = np.empty((len(data), 2), dtype=int)
digit_counts

array([[ 2329281613512397902,  8606218498465358179],
       [ 3473743367810592800,  8319637388744863801],
       [ 7309377861439332468,  6074828605956628594],
       ...,
       [ 2830829216167260783,  2336361472229141612],
       [ 8102831641014329708,  2325341002271386977],
       [ 7955925922710056549, -4458437782395654284]])

In [64]:
for i, line in enumerate(data):
    case, message = line.split("\t")
    num_digits = sum(c.isdigit() for c in message)
    digit_counts[i, 0] = 0 if case == "ham" else 1
    digit_counts[i, 1] = num_digits

#### Here's how the above code works.

Line 8: Loop over data. You use enumerate() to put the value from the list in line and create an index i for this list. To learn more about enumerate(), check out Python enumerate(): Simplify Looping With Counters.

Line 9: Split the line on the tab character to create case and message. case is a string that says whether the message is ham or spam, while message is a string with the text of the message.

Line 10: Calculate the number of digits in the message by using the sum() of a comprehension. In the comprehension, you check each character in the message using isdigit(), which returns True if the element is a numeral and False otherwise. sum() then treats each True result as a 1 and each False as a 0. So, the result of sum() on this comprehension is the number of characters for which isdigit() returned True.

Line 11: Assign values into digit_counts. You assign the first column of the i row to be 0 if the message was legitimate (ham) or 1 if the message was spam.

Line 12: Assign values into digit_counts. You assign the second column of the i row to be the number of digits in the message.

In [65]:
unique_counts = np.unique(digit_counts[:, 1], return_counts=True)
# create an array where the first column has the number of digits in a message, and 
# the second column is the number of messages that have that number of digits

In [66]:
unique_counts = np.transpose(np.vstack(unique_counts))

In [67]:
whitened_counts = whiten(unique_counts)
codebook, _ = kmeans(whitened_counts, 3)

You use whiten() to normalize each feature to have unit variance, which improves the results from kmeans(). Then, kmeans() takes the whitened data and the number of clusters to create as arguments. In this example, you want to create 3 clusters, for definitely ham, definitely spam, and unknown. kmeans() returns two values:

An array with three rows and two columns representing the centroids of each group: The kmeans() algorithm calculates the optimal location of the centroid of each cluster by minimizing the distance from the observations to each centroid. This array is assigned to codebook.

The mean Euclidian distance from the observations to the centroids: You won’t need that value for the rest of this example, so you can assign it to _.

In [68]:
codes, _ = vq(whitened_counts, codebook)

In [69]:
ham_code = codes[0]
spam_code = codes[-1]
unknown_code = list(set(range(3)) ^ set((ham_code, spam_code)))[0]

In [70]:
print("definitely ham:", unique_counts[codes == ham_code][-1])
print("definitely spam:", unique_counts[codes == spam_code][-1])
print("unknown:", unique_counts[codes == unknown_code][-1])

definitely ham: [   0 4110]
definitely spam: [47  1]
unknown: [20 18]


In [72]:
digits = digit_counts[:, 1]
predicted_hams = digits == 0
predicted_spams = digits > 20
predicted_unknowns = np.logical_and(digits > 0, digits <= 20)

In [73]:
spam_cluster = digit_counts[predicted_spams]
ham_cluster = digit_counts[predicted_hams]
unk_cluster = digit_counts[predicted_unknowns]

In [74]:
print("hams:", np.unique(ham_cluster[:, 0], return_counts=True))
print("spams:", np.unique(spam_cluster[:, 0], return_counts=True))
print("unknowns:", np.unique(unk_cluster[:, 0], return_counts=True))

hams: (array([0, 1]), array([4071,   39]))
spams: (array([0, 1]), array([  1, 232]))
unknowns: (array([0, 1]), array([755, 476]))


From this output, you can see that 4110 messages fell into the definitely ham group, of which 4071 were actually ham and only 39 were spam. Conversely, of the 233 messages that fell into the definitely spam group, only 1 was actually ham and the rest were spam.

Of course, over 1200 messages fell into the unknown category, so some more advanced analysis would be needed to classify those messages. You might want to look into something like natural language processing to help improve the accuracy of your prediction, and you can use Python and Keras to help out.